# 10.1 SQL이란
- 스파크는 ANSI SQL:2003의 일부를 구현하였음
    - ANSI SQL:2003은 대부분의 SQL 데이터베이스에서 채택하고 있는 표준
- 그러므로 스파크는 유명한 벤치마크인 TPC-DS를 성공적으로 통과할 수 있음

# 10.2 빅데이터와 SQL: 아파치 하이브
- 스파크 등장 전에는 하이브가 빅데이터 SQL 접근 계층에서 사실상의 표준이었음
    - 페이스북에서 최초로 개발
    - SQL 처리가 필요한 빅데이터 업계에서 믿을 수 없을 정도로 인기 있는 도구가 되었음
- 이제는 많은 사용자가 스파크 SQL을 사용

# 10.3 빅데이터와 SQL: 스파크 SQL
- 스파크 2.0 버전에는 하이브를 지원할 수 있는 상위 호환 기능으로 ANSI-SQL과 HiveQL을 모두 지원한느 자체 개발된 SQL 파서가 포함되어 있음
- 스파크 SQL은 DataFrame과의 뛰어난 호환성 덕분에 다양한 기업에서 강력한 기능으로 자리매김

## 10.3.1 스파크와 하이브의 관계
- 스파크 SQL은 하이브 메타스토어를 사용하므로 하이브와 잘 연동할 수 있음
    - 하이브 메타스토어는 여러 세션에서 사용할 테이블 정보를 보관하고 있음
    - 스파크 SQL은 하이브 메타스토어에 접속(이미 하이브를 사용하고 있는 경우)한 뒤 조회할 파일 수를 최소화하기 위해 메타데이터를 참조
    - 이 기능은 기존 하둡 환경의 모든 워크로드를 스파크로 이관하려는 사용자들에게 인기

### 하이브 메타스토어
- 하이브 메타스토어에 접속하기 위한 몇 가지 속성
    - 접근하려는 하이브 메타스토어에 적합한 버전을 spark.sql.hive.metastore.version에 설정
    - HiveMetastoreClient가 초기화되는 방식을 변경하려면 spark.sql.hive.metastore.jars를 설정
    - 하이브 메타스토어가 저장된 다른 데이터베이스에 접속하려면 적합한 클래스 접두사(MySQL 사용하려면 com.mysql.jdbc로 명시)를 정의해야 함
    - 스파크와 하이브에서 공유할 수 있도록 클래스 접두사를 spark.sql.hive.metastore.sharedPrefixes 속성에 설정

# 10.4 스파크 SQL 쿼리 실행 방법

## 10.4.1 스파크 SQL CLI

## 10.4.2 스파크의 프로그래밍 SQL 인터페이스

In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.2:4040
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1679646846724)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3aaf14d6


In [2]:
spark.sql("SELECT 1+1").show()

+-------+
|(1 + 1)|
+-------+
|      2|
+-------+



- 프로그래밍 방식으로 평가할 수 있는 DataFrame을 반환
- 다른 트랜스포메이션과 마찬가지로 즉시 실행되지 않고 지연 처리됨

- SQL과 DataFrame은 완벽하게 연동될 수 있으므로 더 강력함
    - 예를 들어 DataFrame을 생성하고 SQL을 사용해 처리할 수 있으며 그 결과를 다시 DataFrame으로 돌려받게 됨

In [3]:
spark.read.json("/Users/choyubin/Downloads/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json")
.createOrReplaceTempView("some_sql_view")

In [4]:
spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count)
FROM some_sql_view GROUP BY DEST_COUNTRY_NAME
""")
.where("DEST_COUNTRY_NAME like 'S%'").where("`sum(count)` > 10")
.count()

res3: Long = 12


## 10.4.3 스파크 SQL 쓰리프트 JDBC/ODBC 서버
- 스파크는 자바 데이터베이스 연결(Java Database Connectivity, JDBC) 인터페이스를 제공
- 사용자나 원격 프로그램은 스파크 SQL을 실행하기 위해 이 인터페이스로 스파크 드라이버에 접속
    - e.g. 비즈니스 분석가가 태블로 같은 비즈니스 인텔리전스 소프트웨어를 이용해 스파크에 접속하는 형태

# 10.5 카탈로그
- 스파크 SQL에서 가장 높은 추상화 단계
- 테이블에 저장된 데이터에 대한 메타데이터뿐만 아니라 데이터베이스, 테이블, 함수 그리고 뷰에 대한 정보를 추상화

# 10.6 테이블
- 스파크 SQL을 사용해 유용한 작업을 수행하려면 먼저 테이블을 정의해야 함
- DataFrame과 논리적으로 동일

## 10.6.1 스파크 관리형 테이블
- 관리형 테이블과 외부 테이블 개념은 반드시 기억
- 테이블은 두 가지 중요한 정보를 저장
    - 테이블의 데이터
    - 테이블에 대한 데이터, 즉 메타데이터
- 스파크는 데이터뿐만 아니라 파일에 대한 메타데이터를 관리할 수 있음
- 디스크에 저장된 파일을 이용해 테이블을 정의하면 외부 테이블을 정의하는 것
- DataFrame의 saveAsTable 메서드는 스파크가 관련된 모든 정보를 추적할 수 있는 관리형 테이블을 만들 수 있음
    - saveAsTable 메서드는 테이블을 읽고 데이터를 스파크 포맷으로 변환한 후 새로운 경로에 저장

## 10.6.2 테이블 생성하기

In [5]:
spark.sql("""CREATE TABLE flights_csv(DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING COMMENT "remember, the US will be most prevalent", count LONG)
USING csv OPTIONS (header true, path '/Users/choyubin/Downloads/Spark-The-Definitive-Guide-master/data/flight-data/csv/2015-summary.csv')""").show()

++
||
++
++



In [6]:
spark.sql("""CREATE TABLE flights (
DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
USING JSON OPTIONS (path '/Users/choyubin/Downloads/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json')""").show()

++
||
++
++



In [7]:
spark.sql("""CREATE TABLE flights_from_select USING parquet AS SELECT * FROM flights""")

org.apache.spark.sql.AnalysisException:  Can not create the managed table('`default`.`flights_from_select`'). The associated location('file:/Users/choyubin/Documents/spark/spark-3.3.2-bin-hadoop3/spark-warehouse/flights_from_select') already exists.

## 10.6.4 테이블에 데이터 삽입하기
- 데이터 삽입은 표준 SQL 문법을 따름

In [9]:
spark.sql("""INSERT INTO flights_from_select
SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 20""")

res8: org.apache.spark.sql.DataFrame = []


## 10.6.5 테이블 메타데이터 확인하기
- 테이블 생성 시 코멘트를 추가할 수 있음. 추가된 코멘트를 확인하려면 DESCRIBE 구문을 사용

In [9]:
spark.sql("""DESCRIBE TABLE flights_csv""").show()

+-------------------+---------+--------------------+
|           col_name|data_type|             comment|
+-------------------+---------+--------------------+
|  DEST_COUNTRY_NAME|   string|                null|
|ORIGIN_COUNTRY_NAME|   string|remember, the US ...|
|              count|   bigint|                null|
+-------------------+---------+--------------------+



## 10.6.6 테이블 메타데이터 갱신하기
- REFRESH TABLE
    - 테이블과 관련된 모든 캐싱된 항목을 갱신
- REPAIR TABLE
    - 새로운 파티션 정보를 수집하는 데 초점

In [10]:
spark.sql("""
REFRESH flight_csv""").show()

++
||
++
++



## 10.6.7 테이블 제거하기
- delete(삭제) 불가
- drop(제거)만 가능

In [11]:
spark.sql("""
DROP TABLE flights_csv""")

res10: org.apache.spark.sql.DataFrame = []


In [12]:
spark.sql("""
DROP TABLE IF EXISTS flights_csv""")

res11: org.apache.spark.sql.DataFrame = []


# 10.7 뷰


## 10.7.1 뷰 생성하기

In [13]:
val sql1 = """
CREATE VIEW just_usa_view AS
SELECT * FROM flights WHERE dest_country_name = 'United States'
"""

sql1: String =
"
CREATE VIEW just_usa_view AS
SELECT * FROM flights WHERE dest_country_name = 'United States'
"


In [14]:
spark.sql(sql1)

res12: org.apache.spark.sql.DataFrame = []


In [15]:
val sql2 = """
CREATE TEMP VIEW just_usa_view_temp AS
SELECT * FROM flights WHERE dest_country_name = 'United States'
"""

sql2: String =
"
CREATE TEMP VIEW just_usa_view_temp AS
SELECT * FROM flights WHERE dest_country_name = 'United States'
"


In [16]:
spark.sql(sql2)

res13: org.apache.spark.sql.DataFrame = []


In [17]:
val sql3 = """
CREATE GLOBAL TEMP VIEW just_usa_global_view_temp AS
SELECT * FROM flights WHERE dest_country_name = 'United States'
"""

sql3: String =
"
CREATE GLOBAL TEMP VIEW just_usa_global_view_temp AS
SELECT * FROM flights WHERE dest_country_name = 'United States'
"


In [18]:
spark.sql(sql3)

res14: org.apache.spark.sql.DataFrame = []


In [19]:
spark.sql("""
SHOW TABLES""")

res15: org.apache.spark.sql.DataFrame = [namespace: string, tableName: string ... 1 more field]


In [20]:
val sql4 = """
CREATE OR REPLACE TEMP VIEW just_usa_view_temp AS
SELECT * FROM flights WHERE dest_country_name = 'United States'
"""

sql4: String =
"
CREATE OR REPLACE TEMP VIEW just_usa_view_temp AS
SELECT * FROM flights WHERE dest_country_name = 'United States'
"


In [21]:
spark.sql(sql4)

res16: org.apache.spark.sql.DataFrame = []


In [22]:
spark.sql("""
SELECT * FROM just_usa_view_temp
""")

res17: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


## 10.7.2 뷰 제거하기

In [23]:
val sql5 = """
DROP VIEW IF EXISTS just_usa_view;
"""

sql5: String =
"
DROP VIEW IF EXISTS just_usa_view;
"


In [24]:
spark.sql(sql5)

res18: org.apache.spark.sql.DataFrame = []


# 10.8 데이터베이스
- 데이터베이스를 정의하지 않으면 스파크는 기본 데이터베이스를 사용

In [27]:
spark.sql("""
show databases
""").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



## 10.8.1 데이터베이스 생성하기

In [28]:
spark.sql("""
create database some_db
""")

res21: org.apache.spark.sql.DataFrame = []


## 10.8.2 데이터베이스 설정하기

In [29]:
spark.sql("""
use some_db
""")

res22: org.apache.spark.sql.DataFrame = []


In [30]:
spark.sql("""
SHOW tables""").show()

+---------+------------------+-----------+
|namespace|         tableName|isTemporary|
+---------+------------------+-----------+
|         |just_usa_view_temp|       true|
|         |     some_sql_view|       true|
+---------+------------------+-----------+



In [31]:
spark.sql("""
SELECT * FROM default.flights""").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [32]:
spark.sql("""
select current_database()
""").show()

+------------------+
|current_database()|
+------------------+
|           some_db|
+------------------+



## 10.8.3 데이터베이스 제거하기

In [33]:
spark.sql("""
drop database if exists some_db;
""")

res26: org.apache.spark.sql.DataFrame = []


# 10.10 고급 주제

## 10.10.1 복합 데이터 타입
- 표준 SQL에는 존재하지 않는 매우 강력한 기능
- 구조체, 리스트, 맵

### 구조체
- 스파크에서 중첩 데이터를 생성하거나 쿼리하는 방법을 제공
- 구조체를 만들기 위해서는 여러 컬럼이나 표현식을 괄호로 묶기만 하면 됨

In [34]:
spark.sql("""
use default
""")

res27: org.apache.spark.sql.DataFrame = []


In [39]:
spark.sql("""
create view if not exists nested_data as
select (dest_country_name, origin_country_name) as country, count from flights
""").show()

++
||
++
++



In [40]:
spark.sql("""
select country.*, count from nested_data""").show()

+-----------------+-------------------+-----+
|dest_country_name|origin_country_name|count|
+-----------------+-------------------+-----+
|             null|               null|   15|
|             null|               null|    1|
|             null|               null|  344|
|             null|               null|   15|
|             null|               null|   62|
|             null|               null|    1|
|             null|               null|   62|
|             null|               null|  588|
|             null|               null|   40|
|             null|               null|    1|
|             null|               null|  325|
|             null|               null|   39|
|             null|               null|   64|
|             null|               null|    1|
|             null|               null|   41|
|             null|               null|   30|
|             null|               null|    6|
|             null|               null|    4|
|             null|               

### 리스트

In [41]:
spark.sql("""
select dest_country_name as new_name, collect_list(count) as flight_counts,
collect_set(origin_country_name) as origin_set
from flights group by dest_country_name
""").show()

+--------------------+-------------+---------------+
|            new_name|flight_counts|     origin_set|
+--------------------+-------------+---------------+
|             Algeria|          [4]|[United States]|
|              Angola|         [15]|[United States]|
|            Anguilla|         [41]|[United States]|
| Antigua and Barbuda|        [126]|[United States]|
|           Argentina|        [180]|[United States]|
|               Aruba|        [346]|[United States]|
|           Australia|        [329]|[United States]|
|             Austria|         [62]|[United States]|
|          Azerbaijan|         [21]|[United States]|
|             Bahrain|         [19]|[United States]|
|            Barbados|        [154]|[United States]|
|             Belgium|        [259]|[United States]|
|              Belize|        [188]|[United States]|
|             Bermuda|        [183]|[United States]|
|             Bolivia|         [30]|[United States]|
|Bonaire, Sint Eus...|         [58]|[United St

In [43]:
spark.sql("""
select dest_country_name, array(1, 2, 3) from flights
""").show()

+--------------------+--------------+
|   dest_country_name|array(1, 2, 3)|
+--------------------+--------------+
|       United States|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|               Egypt|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|          Costa Rica|     [1, 2, 3]|
|             Senegal|     [1, 2, 3]|
|             Moldova|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|              Guyana|     [1, 2, 3]|
|               Malta|     [1, 2, 3]|
|            Anguilla|     [1, 2, 3]|
|             Bolivia|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|             Algeria|     [1, 2, 3]|
|Turks and Caicos ...|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
+--------------------+--------------+
only showing top 20 rows



In [45]:
spark.sql("""
select dest_country_name as new_name, collect_list(count)[0]
from flights group by dest_country_name
""").show()

+--------------------+----------------------+
|            new_name|collect_list(count)[0]|
+--------------------+----------------------+
|             Algeria|                     4|
|              Angola|                    15|
|            Anguilla|                    41|
| Antigua and Barbuda|                   126|
|           Argentina|                   180|
|               Aruba|                   346|
|           Australia|                   329|
|             Austria|                    62|
|          Azerbaijan|                    21|
|             Bahrain|                    19|
|            Barbados|                   154|
|             Belgium|                   259|
|              Belize|                   188|
|             Bermuda|                   183|
|             Bolivia|                    30|
|Bonaire, Sint Eus...|                    58|
|              Brazil|                   853|
|British Virgin Is...|                   107|
|            Bulgaria|            

In [46]:
spark.sql("""
create or replace temp view flights_agg as
select dest_country_name, collect_list(count) as collected_counts
from flights group by dest_country_name
""").show()

++
||
++
++



In [47]:
spark.sql("""
select explode(collected_counts), dest_country_name from flights_agg
""").show()

+---+--------------------+
|col|   dest_country_name|
+---+--------------------+
|  4|             Algeria|
| 15|              Angola|
| 41|            Anguilla|
|126| Antigua and Barbuda|
|180|           Argentina|
|346|               Aruba|
|329|           Australia|
| 62|             Austria|
| 21|          Azerbaijan|
| 19|             Bahrain|
|154|            Barbados|
|259|             Belgium|
|188|              Belize|
|183|             Bermuda|
| 30|             Bolivia|
| 58|Bonaire, Sint Eus...|
|853|              Brazil|
|107|British Virgin Is...|
|  3|            Bulgaria|
|  1|        Burkina Faso|
+---+--------------------+
only showing top 20 rows



## 10.10.2 함수

In [48]:
spark.sql("""
show functions
""").show()

+--------+
|function|
+--------+
|       !|
|      !=|
|       %|
|       &|
|       *|
|       +|
|       -|
|       /|
|       <|
|      <=|
|     <=>|
|      <>|
|       =|
|      ==|
|       >|
|      >=|
|       ^|
|     abs|
|    acos|
|   acosh|
+--------+
only showing top 20 rows



spark.sql("""
show system functions
""").show()

In [50]:
spark.sql("""
show user functions
""").show()

+--------+
|function|
+--------+
+--------+



### 사용자 정의 함수

In [51]:
def power3(number:Double):Double = number * number * number
spark.udf.register("power3", power3(_:Double):Double)

power3: (number: Double)Double
res44: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5099/0x0000000802270620@4440c7a6,DoubleType,List(Some(class[value[0]: double])),Some(class[value[0]: double]),Some(power3),false,true)


In [52]:
spark.sql("""
select count, power3(count) from flights
""").show()

+-----+-------------+
|count|power3(count)|
+-----+-------------+
|   15|       3375.0|
|    1|          1.0|
|  344|  4.0707584E7|
|   15|       3375.0|
|   62|     238328.0|
|    1|          1.0|
|   62|     238328.0|
|  588| 2.03297472E8|
|   40|      64000.0|
|    1|          1.0|
|  325|  3.4328125E7|
|   39|      59319.0|
|   64|     262144.0|
|    1|          1.0|
|   41|      68921.0|
|   30|      27000.0|
|    6|        216.0|
|    4|         64.0|
|  230|     1.2167E7|
|    1|          1.0|
+-----+-------------+
only showing top 20 rows



### 서브쿼리
- 쿼리 안에 쿼리를 지정
- 상호연관 서브쿼리
    - 서브쿼리의 정보를 보완하기 위해 쿼리의 외부 범위에 있는 일부 정보를 사용할 수 있음
- 비상호연관 서브쿼리
    - 외부 범위에 있는 정보를 사용하지 않음
- 조건절 서브쿼리

### 비상호연관 조건절 서브쿼리

In [53]:
val sql11 = """
select dest_country_name from flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5
"""

sql11: String =
"
select dest_country_name from flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5
"


In [54]:
spark.sql(sql11).show()

+-----------------+
|dest_country_name|
+-----------------+
|    United States|
|           Canada|
|           Mexico|
|   United Kingdom|
|            Japan|
+-----------------+



In [55]:
val sql12 = """
select * from flights
where origin_country_name in (select dest_country_name from flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5)
"""

sql12: String =
"
select * from flights
where origin_country_name in (select dest_country_name from flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5)
"


In [56]:
spark.sql(sql12).show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Egypt|      United States|   15|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|             Algeria|      United States|    4|
|Turks and Caicos ...|      United States|  230|
|Saint Vincent and...|      United States|    1|
|               Italy|      United States|  382|
|            Pakistan|      United States|   12|
|             Iceland|      United States|  181|
|    Marshall Islands|      United States|   42|
|          Luxembourg|      United States|  155|
|            Honduras|      United States|  362|
|         The Bahama

### 상호연관 조건절 서브쿼리

In [57]:
val sql13 = """
select * from flights f1
where exists (select 1 from flights f2
where f1.dest_country_name = f2.origin_country_name)
and exists (select 1 from flights f2
where f2.dest_country_name = f1.origin_country_name)
"""

sql13: String =
"
select * from flights f1
where exists (select 1 from flights f2
where f1.dest_country_name = f2.origin_country_name)
and exists (select 1 from flights f2
where f2.dest_country_name = f1.origin_country_name)
"


In [58]:
spark.sql(sql13).show()

+--------------------+--------------------+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+--------------------+--------------------+-----+
|       United States|             Romania|   15|
|       United States|             Croatia|    1|
|       United States|             Ireland|  344|
|               Egypt|       United States|   15|
|       United States|               India|   62|
|       United States|           Singapore|    1|
|       United States|             Grenada|   62|
|          Costa Rica|       United States|  588|
|             Senegal|       United States|   40|
|       United States|        Sint Maarten|  325|
|       United States|    Marshall Islands|   39|
|              Guyana|       United States|   64|
|               Malta|       United States|    1|
|            Anguilla|       United States|   41|
|             Bolivia|       United States|   30|
|       United States|            Paraguay|    6|
|Turks and Caicos ...|       United States|  230|
